Given an m x n integer matrix heightMap representing the height of each unit cell in a 2D elevation map, return the volume of water it can trap after raining.

 

Example 1:


Input: heightMap = [[1,4,3,1,3,2],[3,2,1,3,2,4],[2,3,3,2,3,1]]
Output: 4
Explanation: After the rain, water is trapped between the blocks.
We have two small ponds 1 and 3 units trapped.
The total volume of water trapped is 4.
Example 2:


Input: heightMap = [[3,3,3,3,3],[3,2,2,2,3],[3,2,1,2,3],[3,2,2,2,3],[3,3,3,3,3]]
Output: 10
 

Constraints:

m == heightMap.length
n == heightMap[i].length
1 <= m, n <= 200
0 <= heightMap[i][j] <= 2 * 104

물은 block들 사이에서만 갖친다고 가정

각 Line에 대해서 trapping the rain water 1D 버전 돌리면 될 거 같은데?
1D 버전을 어떻게 DP , heap, Pointer로 푸는 지 체크할 것

In [ ]:
class Solution:
    def trapRainWater(self, heightMap: List[List[int]]) -> int:
        # m * n Matrix
        m = len(heightMap)
        n = len(heightMap[0])
        
        def trap1D(water:list[int]) -> int : # 이거 잘못됨. 맨 밑에 python code : left/right max를 따로 관리해야함
            l = 0
            r = len(water)-1
            ans = 0
            small = min(water[0],water[r])
            while l < r :
                if water[l] <= water[r]:
                    l+=1
                    if water[l] >= small :
                        small = water[l]
                        continue
                    else : ans += (small-water[l])
                else:
                    r-=1
                    if water[r] >= small :
                        small = water[r]
                        continue
                    else : ans += (small-water[r])
            return ans
        curr = 0
        for i in range(n):
            temp = []
            for j in range(m):
                temp.append(heightMap[j][i])
            curr += trap1D(temp)
        return curr

        # Nope.. 가생이 edge 부분 처리 X..


        # 위 코드 기준 모범답안
        """
        class Solution:
    def trapRainWater(self, heightMap: List[List[int]]) -> int:
        # m * n Matrix
        m = len(heightMap)
        n = len(heightMap[0])

        if m < 3 or n < 3:
            return 0  # No water can be trapped in less than 3x3 grid

        def trap1D(water: List[int]) -> int:
            l, r = 0, len(water) - 1
            ans = 0
            left_max, right_max = water[l], water[r]
            while l <= r:
                if left_max <= right_max:
                    if water[l] < left_max:
                        ans += left_max - water[l]
                    else:
                        left_max = water[l]
                    l += 1
                else:
                    if water[r] < right_max:
                        ans += right_max - water[r]
                    else:
                        right_max = water[r]
                    r -= 1
            return ans

        # Initialize visited and priority queue for boundary cells
        visited = [[False] * n for _ in range(m)]
        min_heap = []

        # Add all boundary cells to the heap
        for i in range(m):
            for j in range(n):
                if i == 0 or i == m - 1 or j == 0 or j == n - 1:
                    heapq.heappush(min_heap, (heightMap[i][j], i, j))
                    visited[i][j] = True

        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        water_trapped = 0

        # Process cells in the heap
        while min_heap:
            height, x, y = heapq.heappop(min_heap)
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n and not visited[nx][ny]:
                    # Calculate trapped water
                    water_trapped += max(0, height - heightMap[nx][ny])
                    # Update height and mark as visited
                    heapq.heappush(min_heap, (max(height, heightMap[nx][ny]), nx, ny))
                    visited[nx][ny] = True

        return water_trapped

        """

In [ ]:
# 모범답안
import heapq
from typing import List

class Solution:
    def trapRainWater(self, heightMap: List[List[int]]) -> int:
        if not heightMap or not heightMap[0]:
            return 0
        
        m, n = len(heightMap), len(heightMap[0])
        visited = [[False] * n for _ in range(m)]
        heap = []
        
        # Step 1: Add all boundary cells to the heap
        for i in range(m):
            for j in range(n):
                if i == 0 or j == 0 or i == m - 1 or j == n - 1:
                    heapq.heappush(heap, (heightMap[i][j], i, j)) # MinHeap이용 --> 젤 낮은놈부터 생각하는구나 물 체우기
                    visited[i][j] = True
        
        # Step 2: Process cells in the heap
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        water_trapped = 0
        
        while heap:
            height, x, y = heapq.heappop(heap)
            
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                
                if 0 <= nx < m and 0 <= ny < n and not visited[nx][ny]:
                    # Calculate trapped water at (nx, ny)
                    water_trapped += max(0, height - heightMap[nx][ny])
                    # Update visited cell and push it to the heap
                    heapq.heappush(heap, (max(height, heightMap[nx][ny]), nx, ny))
                    visited[nx][ny] = True
        
        return water_trapped


코드 작동 원리
Step 1: Boundary Cells 추가
heap은 Python의 heapq 모듈을 사용하여 구현된 Min-Heap입니다. 이는 항상 가장 낮은 높이 값을 가진 셀을 우선적으로 처리합니다.
먼저, 모든 가장자리 셀(boundary cells)을 heap에 추가하고, 이 셀들은 이미 방문한 것으로 표시합니다(visited[i][j] = True).
가장자리는 물을 가두는 데 사용되지 않고 물이 흘러나가는 경계 역할을 합니다.
Step 2: Min-Heap 기반 BFS 탐색
Min-Heap에서 가장 낮은 높이의 셀을 꺼내고(heapq.heappop()), 이 셀에 연결된 인접 셀을 탐색합니다.
각 인접 셀이 방문되지 않았다면:
이 셀에서 물을 가둘 수 있는지 계산:
height - heightMap[nx][ny]가 양수라면 물이 갇힐 수 있는 양입니다.
이 셀의 높이를 업데이트:
max(height, heightMap[nx][ny])로 현재 높이와 인접 셀의 높이 중 더 큰 값을 선택합니다. 이렇게 하면 물이 넘치지 않도록 새로운 높이를 설정합니다.
해당 셀을 heap에 추가하고 방문으로 표시합니다.
왜 Min-Heap을 사용하는가?
Min-Heap은 항상 현재 가장 낮은 높이의 셀부터 처리합니다.
낮은 셀부터 처리함으로써, 물이 흘러내려가는 경로를 탐색하며 주변 높이를 업데이트할 수 있습니다.
이 방법은 높이가 높은 셀에서 낮은 셀로 물이 역류하는 상황을 방지합니다.
핵심 로직
Boundary Cells 탐색 초기화:

물은 항상 가장자리에서부터 흘러내립니다. 따라서 모든 가장자리 셀을 힙에 추가하여 탐색의 시작점으로 사용합니다.
우선순위 큐를 사용한 탐색:

힙에서 가장 낮은 높이의 셀을 꺼냅니다.
이 셀을 기준으로 인접 셀의 높이를 확인하고 물이 갇힐 수 있는 양을 계산합니다.
방문 처리:

각 셀은 한 번만 탐색됩니다(visited 배열로 추적). 이를 통해 불필요한 중복 계산을 방지합니다.

In [ ]:
# C
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>

#define MAX_HEAP_SIZE 10000

typedef struct {
    int height, x, y;
} Cell;

typedef struct {
    Cell heap[MAX_HEAP_SIZE];
    int size;
} MinHeap;

void heapPush(MinHeap *h, Cell cell) {
    int i = h->size++;
    while (i > 0) {
        int parent = (i - 1) / 2;
        if (h->heap[parent].height <= cell.height) break;
        h->heap[i] = h->heap[parent];
        i = parent;
    }
    h->heap[i] = cell;
}

Cell heapPop(MinHeap *h) {
    Cell top = h->heap[0];
    Cell temp = h->heap[--h->size];
    int i = 0, child;
    while ((child = 2 * i + 1) < h->size) {
        if (child + 1 < h->size && h->heap[child + 1].height < h->heap[child].height)
            child++;
        if (temp.height <= h->heap[child].height) break;
        h->heap[i] = h->heap[child];
        i = child;
    }
    h->heap[i] = temp;
    return top;
}

int trapRainWater(int** heightMap, int heightMapSize, int* heightMapColSize) {
    if (heightMapSize < 3 || *heightMapColSize < 3) return 0;

    int m = heightMapSize, n = *heightMapColSize;
    bool visited[m][n];
    memset(visited, 0, sizeof(visited));

    MinHeap heap = { .size = 0 };
    int directions[4][2] = { {0, 1}, {1, 0}, {0, -1}, {-1, 0} };
    int waterTrapped = 0;

    for (int i = 0; i < m; i++) {
        for (int j = 0; j < n; j++) {
            if (i == 0 || i == m - 1 || j == 0 || j == n - 1) {
                heapPush(&heap, (Cell){heightMap[i][j], i, j});
                visited[i][j] = true;
            }
        }
    }

    while (heap.size > 0) {
        Cell cell = heapPop(&heap);
        for (int d = 0; d < 4; d++) {
            int nx = cell.x + directions[d][0], ny = cell.y + directions[d][1];
            if (nx >= 0 && nx < m && ny >= 0 && ny < n && !visited[nx][ny]) {
                waterTrapped += (cell.height > heightMap[nx][ny]) ? cell.height - heightMap[nx][ny] : 0;
                heapPush(&heap, (Cell){(cell.height > heightMap[nx][ny]) ? cell.height : heightMap[nx][ny], nx, ny});
                visited[nx][ny] = true;
            }
        }
    }

    return waterTrapped;
}


In [ ]:
# C++
#include <vector>
#include <queue>
#include <tuple> 튜플을 사용하는것
"""
#include <tuple>
#include <iostream>
using namespace std;

int main() {
    // 서로 다른 데이터 타입으로 tuple 생성
    tuple<int, bool, char, string> myTuple = make_tuple(42, true, 'A', "Hello");

    // 각 요소 접근
    cout << "Integer: " << get<0>(myTuple) << endl;  // 42
    cout << "Boolean: " << get<1>(myTuple) << endl;  // 1 (true)
    cout << "Character: " << get<2>(myTuple) << endl;  // A
    cout << "String: " << get<3>(myTuple) << endl;  // Hello

    return 0;
}

"""
using namespace std;

class Solution {
public:
    int trapRainWater(vector<vector<int>>& heightMap) {
        int m = heightMap.size(), n = heightMap[0].size();
        if (m < 3 || n < 3) return 0;

        vector<vector<bool>> visited(m, vector<bool>(n, false));
        priority_queue<tuple<int, int, int>, vector<tuple<int, int, int>>, greater<>> minHeap;

        for (int i = 0; i < m; i++) {
            for (int j = 0; j < n; j++) {
                if (i == 0 || i == m - 1 || j == 0 || j == n - 1) {
                    minHeap.emplace(heightMap[i][j], i, j);
                    visited[i][j] = true;
                }
            }
        }

        int waterTrapped = 0;
        vector<int> dirs = {0, 1, 0, -1, 0};

        while (!minHeap.empty()) {
            auto [height, x, y] = minHeap.top();
            minHeap.pop();
            for (int d = 0; d < 4; d++) {
                int nx = x + dirs[d], ny = y + dirs[d + 1];
                if (nx >= 0 && nx < m && ny >= 0 && ny < n && !visited[nx][ny]) {
                    waterTrapped += max(0, height - heightMap[nx][ny]);
                    minHeap.emplace(max(height, heightMap[nx][ny]), nx, ny);
                    visited[nx][ny] = true;
                }
            }
        }

        return waterTrapped;
    }
};


In [ ]:
# 내가 푼 C++ 어렵다..
class Solution {
public:
    int trapRainWater(vector<vector<int>>& heightMap) {
        int m = heightMap.size();
        int n = heightMap[0].size();

        vector<vector<bool>> visited(m,vector<bool>(n,false));
        priority_queue<tuple<int,int,int>,vector<tuple<int,int,int>>,greater<tuple<int,int,int>> > min_heap;
        
        // fill the pq
        for(int i=0;i<m;i++){
            for(int j=0;j<n;j++){
                if(i == 0 || j == 0 || i == m-1 || j == n-1){
                    visited[i][j] = true;
                    min_heap.push({heightMap[i][j],i,j});
                }
            }
        }
        // initialize direction for matrix(BFS)
        vector<pair<int,int>> dir = {
            {1,0},{-1,0},{0,1},{0,-1}
        };
        int ans = 0;
        tuple<int,int,int> temp;

        while(!min_heap.empty()){
            temp = min_heap.top();
            min_heap.pop();
            int height = get<0>(temp);
            int x = get<1>(temp);
            int y = get<2>(temp);
            for(auto const p : dir){
                int dx = x+p.first;
                int dy = y+p.second;
                if(dx >=0 && dx <m && dy >= 0 && dy < n && !visited[dx][dy] ){
                    ans += max(0,height - heightMap[dx][dy]);
                    min_heap.push({max(height,heightMap[dx][dy]),dx,dy});
                    visited[dx][dy] = true;
                }
            }
        }
        return ans;        
    }
};

In [ ]:
# iD water rain trap
class Solution:
    def trap(self, height: List[int]) -> int:
        l, r = 0, len(height) - 1
        left_max, right_max = height[l], height[r]
        ans = 0

        while l < r:
            if left_max <= right_max:
                l += 1
                if height[l] < left_max:
                    ans += left_max - height[l]
                else:
                    left_max = height[l]
            else:
                r -= 1
                if height[r] < right_max:
                    ans += right_max - height[r]
                else:
                    right_max = height[r]

        return ans